In [1]:
%load_ext lab_black

In [2]:
from pathlib import Path

import contextily as ctx
import geopandas as gpd
from joblib import Parallel, delayed
from shapely.geometry import Polygon

In [3]:
root = Path("..", "data")
out_path = Path(root, "outputs")
img_path = Path(root, "figures")

In [4]:
coastline = gpd.read_file(
    "https://www2.census.gov/geo/tiger/TIGER2020/COASTLINE/tl_2020_us_coastline.zip"
)

east_clip = Polygon(
    [
        (-98.0189, 25.6163),
        (-99.0192, 28.2578),
        (-70.0293, 44.8238),
        (-65.9741, 45.6041),
        (-66.4375, 44.5238),
        (-78.9686, 31.6511),
        (-79.75, 24.4402),
        (-83.5973, 24.1171),
        (-91.1961, 28.1507),
        (-96.3344, 25.6423),
        (-98.0189, 25.6163),
    ]
)

gulf_east = gpd.clip(coastline, east_clip).reset_index(drop=True)
gulf_east.to_file(Path(out_path, "gulf_east_coastline.gpkg"))

In [5]:
gulf_east = (
    gulf_east[~gulf_east.geometry.is_ring].reset_index(drop=True).to_crs("EPSG:3857")
)
bboxes = gulf_east.buffer(5e3)[gulf_east.length > 5e3].bounds

fpath = Path(img_path, "coast_images")
source = ctx.providers.Stamen.TonerBackground
_ = Parallel(n_jobs=12)(
    delayed(ctx.bounds2raster)(
        w, s, e, n, Path(fpath, f"image_{i}.tif"), source=source, zoom=9
    )
    for i, (w, s, e, n) in bboxes.iterrows()
)